## **Proyecto: Marketing bancario**

- **Recurso:** S. Moro, P. Rita, P. Cortez (2012). Los datos están relacionados con campañas de marketing directo (llamadas telefónicas) de una institución bancaria portuguesa. DOI: [10.24432/C5K306](https://archive.ics.uci.edu/dataset/222/bank+marketing)
- **Donado:** 2/13/2012
- **URL:** https://archive.ics.uci.edu/dataset/222/bank+marketing
- **Objetivo:** Es predecir si el cliente suscribirá un depósito a plazo (variable y).

**Paper introductorio**
- Un enfoque basado en datos para predecir el éxito del telemarketing bancario
- Por Sérgio Moro, P. Cortez, P. Rita . 2014
- Publicado en Sistemas de apoyo a la toma de decisiones

**Información adicional**

Los datos están relacionados con campañas de marketing directo de una institución bancaria portuguesa. Las campañas de marketing se basaron en llamadas telefónicas. A menudo, se requirió más de un contacto con el mismo cliente, para acceder si el producto (depósito a plazo bancario) sería ('sí') o no ('no') suscrito. Hay cuatro conjuntos de datos: 
1) bank-additional-full.csv con todos los ejemplos (41188) y 20 entradas, ordenados por fecha (de mayo de 2008 a noviembre de 2010), muy cerca de los datos analizados en [Moro et al., 2014] 
2) bank-additional.csv con el 10% de los ejemplos (4119), seleccionados aleatoriamente de 1), y 20 entradas. 
3) bank-full.csv con todos los ejemplos y 17 entradas, ordenados por fecha (versión anterior de este conjunto de datos con menos entradas). 
4) bank.csv con el 10% de los ejemplos y 17 entradas, seleccionadas aleatoriamente de 3 (versión anterior de este conjunto de datos con menos entradas). Los conjuntos de datos más pequeños se proporcionan para probar algoritmos de aprendizaje automático que requieren más recursos computacionales (por ejemplo, SVM). El objetivo de la clasificación es predecir si el cliente suscribirá (sí/no) un depósito a plazo (variable y).

**License**

Este conjunto de datos está licenciado bajo una licencia [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/legalcode) (CC BY 4.0).

Esto permite compartir y adaptar los conjuntos de datos para cualquier propósito, siempre que se otorgue el crédito correspondiente.

In [1]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler)
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Decission Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree


from sklearn.metrics import (accuracy_score,
                            confusion_matrix,
                            classification_report)

import requests
from io import StringIO

## **1. Cargar datos**
**Objetivo:** Obtener los datos de la fuente y una primera visión de las propiedades, más una presentación.

In [2]:
df_raw = pd.read_csv('../data/raw_2/bank-full.csv', sep=';')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [3]:
df_raw.sample(10, random_state=42)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown,no
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown,no
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown,no
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure,no
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown,no
30813,28,blue-collar,married,secondary,no,1134,no,no,cellular,9,feb,130,3,-1,0,unknown,no
35463,24,management,single,tertiary,no,1085,no,yes,cellular,7,may,95,6,-1,0,unknown,no
31382,37,admin.,single,secondary,no,127,no,no,cellular,23,mar,83,4,-1,0,unknown,no
16904,30,blue-collar,single,secondary,no,3,yes,no,cellular,25,jul,51,1,-1,0,unknown,no
11930,38,technician,single,secondary,no,258,no,yes,unknown,20,jun,587,2,-1,0,unknown,no


## **Diccionario de datos**

|Nombre de la variable|	Role|	Tipo|	Demográfico|	Descripción|	Unidades|	Valores faltantes|
|---------------------|-----|-------|--------------|---------------|------------|--------------------|
|edad|	Característica|	Entero|	Edad|	|	|	 No|
|trabajo|	Característica|	Categórico|	Ocupación|	tipo de trabajo (categórico: 'administrador', 'obrero', 'empresario', 'empleado doméstico', 'gerente', 'jubilado', 'autónomo', 'servicios', 'estudiante', 'técnico', 'desempleado', 'desconocido')|	|	No|
|marital|	Característica|	Categórico|	Estado civil|	estado civil (categórico: 'divorciado', 'casado', 'soltero', 'desconocido'; nota: 'divorciado' significa divorciado o viudo)|	|	No|
|educación|	Característica|	Categórico|	Nivel de educación|	(categórico: 'básico.4a', 'básico.6a', 'básico.9a', 'bachillerato', 'analfabeto', 'curso profesional', 'título universitario', 'desconocido')|	|	No|
|por defecto|	Característica|	Binario|	|	¿tiene crédito en mora?|	|	No|
|balance|	Característica|	Entero|	|	saldo medio anual|	euros |	No|
|alojamiento|	Característica|	Binario|	|	¿Tiene préstamo de vivienda?|	|	No|
|préstamo|	Característica|	Binario|	|	¿Tiene préstamo personal?|	|	No|
|contacto|	Característica|	Categórico|	|Tipo de comunicación de contacto (categórica: 'celular', 'teléfono')|	|	Sí|
|día_de_la_semana|	Característica|	Fecha|	|	último día de contacto de la semana|	|	No|
|mes|	Característica|	Fecha|	|	último mes de contacto del año (categórico: 'jan', 'feb', 'mar', ..., 'nov', 'dic')|	|	No|
|duración|	Característica|	Entero|	|	Duración del último contacto, en segundos (numérico). Nota importante: este atributo afecta en gran medida el resultado final (por ejemplo, si la duración es 0, entonces y es "no"). Sin embargo, la duración no se conoce antes de que se realice una llamada. Además, después del final de la llamada, obviamente se conoce y. Por lo tanto, esta entrada solo se debe incluir con fines de referencia y se debe descartar si la intención es tener un modelo predictivo realista.|	|	No|
|campaña|	Característica|	Entero|	|	Número de contactos realizados durante esta campaña y para este cliente (numérico, incluye el último contacto)|	|	No|
|días de paz|	Característica|	Entero|	|	Número de días transcurridos desde que se contactó por última vez al cliente de una campaña anterior (numérico; -1 significa que el cliente no fue contactado previamente)|	|	Sí|
|anterior|	Característica|	Entero| |	Número de contactos realizados antes de esta campaña y para este cliente.|	|	No|
|puchero|	Característica|	Categórico|	|	resultado de la campaña de marketing anterior (categórico: 'fracaso', 'inexistente', 'éxito')|	|	Sí|
|y|	Objetivo|	Binario|	|	¿El cliente ha suscrito un depósito a plazo?|	|	No|

## **2. Preprocesamiento de datos**

**Objetivo:** Realizar los pasos de limpieza de datos, transformación de datos y reducción de datos para evitar la falta de coincidencia de datos, datos ruidosos o datos no ordenados.